# Amazonia-1 on AWS Open Data

## Accessing Cloud Optimized Multi-Band Raster Data


With the AWS ASDI STAC API we can search and load various geospatial datasets. Below we'll work with the Amazonia-1 Satelite WFI Camera, Level-4 Orthorectified data, sourced from Cloud Optimized GeoTIFFs (COGs) [hosted on AWS](https://registry.opendata.aws/amazonia/). In this example, we read in 4 band imagery within a bounding box.

First we install any necessary packages. Please note that the following block of code only needs to be run if needed within a new workspace and that you may need to restart the kernel to use updated packages. The code block only needs to be run once for each AWS Sagemaker Studio profile.

In [1]:
%pip install --upgrade pip -q
%pip install matplotlib 'pystac_client>=0.5.0' stackstac rasterio folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


We'll use `pystac_client` to submit our search query, stackstac to load and composite our rasters as an Xarray DataArray, and rio-tiler to performantally load individual COGs.

### Querying the AWS STAC API with Pystac Client

First, we define the URL to our STAC API, a service that allows us to submit search queries for spatio-temporal assets (like elevation raster tiles).

Next, we open this URL with `pystac_client`, which allows us to use the client's search method. `client.search` accepts many different arguments to specify the search query. Common query parameters are `time_range` and `bbox` (the spatial bounding box). Since the DEM was collected at one point in time, we will only specify the "where" with a `bbox` describing the minx, miny, maxx, maxy coordinates in lat/lon degrees.

In [2]:
import os
from pystac_client import Client
import stackstac
# making a connection to AWS STAC API
API_ROOT_URL = "https://dev.asdi-catalog.org"
client = Client.open(API_ROOT_URL)

# our search requirements, a bounding box for San Ignacio. Belize
#bbox = [-89.19605011359278, 17.028707271735962, -88.94369134663197, 17.263285211322446]
# try Curitiba Brazil
bbox = [-49.320232, -25.626009, -48.927635, -25.323504]
source_search = client.search(
        collections=["AMAZONIA1-WFI"], 
        bbox=bbox,
        max_items=100
    )
# getting the metadata and links to data assets in STAC format
source_items = source_search.item_collection()
print("Number of WFI tiles: ", len(source_items))

Number of WFI tiles:  15


### Loading an OpenStreetMap(OSM) descriptive map of the same area

To better understand our area of interest, we can create a descriptive map with our bounding box. We'll load an OpenStreetMap basemap that includes place names, annotations for water and forested areas, and roads. We'll use `folium` to access and plot this map. To learn more about `folium`, check out the [documentation](https://python-visualization.github.io/folium/). We'll use our bounding box to specify where we want to center our map and pick a zoom level that shows us detail around San Ignacio.

In [14]:
# import shapely for manipulation and analysis of geometric objects
# in this case to find the center of the bounding box bbox
import shapely as shp
bbox_poly = shp.geometry.box(*bbox)
center_point = (bbox_poly.centroid)

In [13]:
import folium
m = folium.Map(location=[center_point.y, center_point.x], zoom_start=9)
bbox_style_blue = {'fillColor': '#3388ff', 'color': '#3388ff', 'fillOpacity':'0.05'} #make bbox red
folium.GeoJson(source_items.to_dict(), name="items", style_function=lambda x:bbox_style_blue).add_to(m)
bbox_style_red = {'fillColor': '#ff0000', 'color': '#ff0000'} #make bbox red
folium.GeoJson(bbox_poly, name="bbox", style_function=lambda x:bbox_style_red).add_to(m)
m

### Loading DEM raster into an Xarray DataArray

There are multiple options for loading STAC items into in-memory arrays. `stackstac` allows you to specify a list of items from a `pystac_client` search and load those items as a single xarray DataArray. Similar to it's name, `stackstac` handles loading and then stacking individual COGs that fall within our `pystac` query.

We have 4 bands and multiple time samples over the same area.


In [25]:
# We have data in 2 different projections
# TODO: remove this block, or use it to decide which epsg to use.
for item in source_items:
    print(item.properties.get('proj:epsg'))

32723
32722
32723
32722
32723
32723
32722
32722
32723
32722
32723
32722
32723
32722
32723


In [7]:
# rio cogeo info was used to determine the resolution of the data
# feedback when using STAC projection extension include transform
# Load an Xarray DataArray with a stack of scenes and bands
# Specify the projection since 2 are present in the source
# Specify the bbox to limit the data read
stack = stackstac.stack(
    source_items, 
    resolution=64, 
    epsg=32723, 
    bounds_latlon=bbox)

/home/studio-lab-user/.conda/envs/dem/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [8]:
# Look at the structure of the data cube
# 15 scenes, each scene is 4 bands, with a resolution of 635x545
stack

<xarray.DataArray 'stackstac-d312ff8046a8586e42b93cf60675dd4e' (time: 15,
                                                                band: 4,
                                                                y: 545, x: 635)>
dask.array<fetch_raster_window, shape=(15, 4, 545, 635), dtype=float64, chunksize=(1, 1, 545, 635), chunktype=numpy.ndarray>
Coordinates: (12/21)
  * time                                   (time) datetime64[ns] 2023-06-18T1...
    id                                     (time) <U34 'AMAZONIA_1_WFI_202306...
  * band                                   (band) <U2 'B1' 'B2' 'B3' 'B4'
  * x                                      (x) float64 6.49e+04 ... 1.055e+05
  * y                                      (y) float64 7.193e+06 ... 7.159e+06
    view:off_nadir                         (time) float64 0.0002134 ... 0.004034
    ...                                     ...
    sat:orbit_state                        <U10 'descending'
    amazonia:path                          (time) int64 34 35 34 35 ... 34 35 34
    common_name                            (band) <U5 'blue' 'green' 'red' 'nir'
    center_wavelength                      object None
    full_width_half_max                    object None
    epsg                                   int64 32723
Attributes:
    spec:        RasterSpec(epsg=32723, bounds=(64896, 7158592, 105536, 71934...
    crs:         epsg:32723
    transform:   | 64.00, 0.00, 64896.00|\n| 0.00,-64.00, 7193472.00|\n| 0.00...
    resolution:  64

In [27]:
source_items[3].properties

{'gsd': 64.0,
 'datetime': '2023-07-20T13:46:39Z',
 'platform': 'amazonia-1',
 'proj:epsg': 32722,
 'instruments': ['WFI'],
 'amazonia:row': 19,
 'amazonia:path': 35,
 'view:off_nadir': 0.000144213,
 'sat:orbit_state': 'descending',
 'view:sun_azimuth': 29.45145,
 'amazonia:data_type': 'L4',
 'view:sun_elevation': 40.092349999999996,
 'sat:platform_international_designator': '2021-015A'}

In [ ]:
## Compare Scenes

- List the date of each scene
- Pick 2 times
- Plot False Color IR
- Plot them side by side
- Plot them in an Interactive map?